## List All Feature Views

# Feature Engineering with Snowflake Feature Store

This notebook derives features from raw continuous customer data using **Snowflake Feature Store**.

The Feature Store provides:
- Centralized feature definitions and management
- Automatic feature refresh on schedule
- Point-in-time correct features for training
- Feature lineage and discovery
- Consistent features for training and inference

**Prerequisites**: Run `generate_continuous_data.ipynb` first to create raw data tables.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F, Window
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)

session = get_active_session()

## Configuration

In [ ]:
DATABASE = 'ML_DEMO'
SCHEMA = 'PUBLIC'
FEATURE_STORE_NAME = 'CLV_FEATURE_STORE'
WAREHOUSE = 'ML_DEMO_WH'

session.use_database(DATABASE)
session.use_schema(SCHEMA)
session.use_warehouse(WAREHOUSE)

OBSERVATION_DATE = datetime(2024, 6, 30)

print(f"Database: {DATABASE}")
print(f"Schema: {SCHEMA}")
print(f"Feature Store: {FEATURE_STORE_NAME}")
print(f"Warehouse: {WAREHOUSE}")
print(f"Observation Date: {OBSERVATION_DATE}")

## Create or Connect to Feature Store

A feature store in Snowflake is a schema that contains feature views (backed by dynamic tables or views).

In [ ]:
fs = FeatureStore(
    session=session,
    database=DATABASE,
    name=FEATURE_STORE_NAME,
    default_warehouse=WAREHOUSE,
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

print(f"✓ Feature Store ready: {DATABASE}.{FEATURE_STORE_NAME}")

## Register Entity

Entities organize features by subject. Here we create a CUSTOMER entity with CUSTOMER_ID as the join key.

In [ ]:
try:
    customer_entity = fs.get_entity("CUSTOMER")
    print("✓ CUSTOMER entity already exists")
except:
    customer_entity = Entity(
        name="CUSTOMER",
        join_keys=["CUSTOMER_ID"],
        desc="Customer entity for CLV prediction"
    )
    fs.register_entity(customer_entity)
    print("✓ Created CUSTOMER entity")

print(f"  Join keys: {customer_entity.join_keys}")

## Feature View 1: RFM Features

**RFM** (Recency, Frequency, Monetary) features capture customer purchase behavior:
- **Recency**: Days since last purchase
- **Frequency**: Total number of purchases  
- **Monetary**: Total and average spending

In [ ]:
transactions_df = session.table(f"{DATABASE}.{SCHEMA}.CONTINUOUS_TRANSACTIONS")

rfm_df = transactions_df.group_by("CUSTOMER_ID").agg([
    F.datediff(
        "day",
        F.max("TRANSACTION_DATE"),
        F.lit(OBSERVATION_DATE)
    ).alias("RECENCY_DAYS"),
    F.count("TRANSACTION_ID").alias("FREQUENCY"),
    F.sum("AMOUNT").alias("MONETARY_TOTAL"),
    F.avg("AMOUNT").alias("MONETARY_AVG"),
    F.min("TRANSACTION_DATE").alias("FIRST_PURCHASE_DATE"),
    F.max("TRANSACTION_DATE").alias("LAST_PURCHASE_DATE")
])

rfm_df = rfm_df.with_column(
    "CUSTOMER_TENURE_DAYS",
    F.datediff("day", F.col("FIRST_PURCHASE_DATE"), F.lit(OBSERVATION_DATE))
)

print("RFM feature DataFrame:")
rfm_df.show(5)

In [ ]:
rfm_fv = FeatureView(
    name="RFM_FEATURES",
    entities=[customer_entity],
    feature_df=rfm_df,
    refresh_freq="1 day",
    desc="Recency, Frequency, Monetary features from transaction history"
).attach_feature_desc({
    "RECENCY_DAYS": "Days since last purchase (lower = more recent)",
    "FREQUENCY": "Total number of purchases (count of transactions)",
    "MONETARY_TOTAL": "Total amount spent across all transactions",
    "MONETARY_AVG": "Average transaction amount",
    "CUSTOMER_TENURE_DAYS": "Days since first purchase (customer age)"
})

rfm_fv_registered = fs.register_feature_view(
    feature_view=rfm_fv,
    version="1.0",
    block=True
)

print("✓ Registered RFM_FEATURES feature view")

## Feature View 2: Purchase Pattern Features

Advanced behavioral features:
- Inter-purchase time patterns
- Product category diversity
- Recent activity windows (30d, 90d)
- Spending trends

In [ ]:
customers_df = session.table(f"{DATABASE}.{SCHEMA}.CONTINUOUS_CUSTOMERS_PROFILE")
transactions_df = session.table(f"{DATABASE}.{SCHEMA}.CONTINUOUS_TRANSACTIONS")

purchase_patterns_df = transactions_df.group_by("CUSTOMER_ID").agg([
    F.count_distinct("PRODUCT_CATEGORY").alias("UNIQUE_CATEGORIES_PURCHASED"),
    F.sum("QUANTITY").alias("TOTAL_ITEMS_PURCHASED"),
    F.sum(
        F.when(
            F.col("TRANSACTION_DATE") >= F.dateadd("day", F.lit(-30), F.lit(OBSERVATION_DATE)),
            F.col("AMOUNT")
        ).otherwise(F.lit(0))
    ).alias("RECENT_30D_AMOUNT"),
    F.sum(
        F.when(
            F.col("TRANSACTION_DATE") >= F.dateadd("day", F.lit(-30), F.lit(OBSERVATION_DATE)),
            F.lit(1)
        ).otherwise(F.lit(0))
    ).alias("RECENT_30D_COUNT"),
    F.sum(
        F.when(
            F.col("TRANSACTION_DATE") >= F.dateadd("day", F.lit(-90), F.lit(OBSERVATION_DATE)),
            F.col("AMOUNT")
        ).otherwise(F.lit(0))
    ).alias("RECENT_90D_AMOUNT"),
    F.sum(
        F.when(
            F.col("TRANSACTION_DATE") >= F.dateadd("day", F.lit(-90), F.lit(OBSERVATION_DATE)),
            F.lit(1)
        ).otherwise(F.lit(0))
    ).alias("RECENT_90D_COUNT")
])

print("Purchase patterns feature DataFrame:")
purchase_patterns_df.show(5)

In [ ]:
purchase_patterns_fv = FeatureView(
    name="PURCHASE_PATTERNS",
    entities=[customer_entity],
    feature_df=purchase_patterns_df,
    refresh_freq="1 day",
    desc="Purchase behavior patterns and trends"
).attach_feature_desc({
    "UNIQUE_CATEGORIES_PURCHASED": "Number of distinct product categories purchased",
    "TOTAL_ITEMS_PURCHASED": "Total quantity of items purchased",
    "RECENT_30D_AMOUNT": "Total spending in last 30 days",
    "RECENT_30D_COUNT": "Number of transactions in last 30 days",
    "RECENT_90D_AMOUNT": "Total spending in last 90 days",
    "RECENT_90D_COUNT": "Number of transactions in last 90 days"
})

purchase_patterns_fv_registered = fs.register_feature_view(
    feature_view=purchase_patterns_fv,
    version="1.0",
    block=True
)

print("✓ Registered PURCHASE_PATTERNS feature view")

## Feature View 3: Engagement Features

Non-purchase engagement signals:
- Website visits
- Email interactions
- Support tickets
- Product views and cart adds

In [ ]:
interactions_df = session.table(f"{DATABASE}.{SCHEMA}.CONTINUOUS_INTERACTIONS")

engagement_df = interactions_df.group_by("CUSTOMER_ID").agg([
    F.count("INTERACTION_ID").alias("TOTAL_INTERACTIONS"),
    F.sum(
        F.when(F.col("EVENT_TYPE") == F.lit("website_visit"), F.lit(1))
        .otherwise(F.lit(0))
    ).alias("WEBSITE_VISITS"),
    F.sum(
        F.when(F.col("EVENT_TYPE") == F.lit("email_open"), F.lit(1))
        .otherwise(F.lit(0))
    ).alias("EMAIL_OPENS"),
    F.sum(
        F.when(F.col("EVENT_TYPE") == F.lit("email_click"), F.lit(1))
        .otherwise(F.lit(0))
    ).alias("EMAIL_CLICKS"),
    F.sum(
        F.when(F.col("EVENT_TYPE") == F.lit("support_ticket"), F.lit(1))
        .otherwise(F.lit(0))
    ).alias("SUPPORT_TICKETS"),
    F.sum(
        F.when(F.col("EVENT_TYPE") == F.lit("product_view"), F.lit(1))
        .otherwise(F.lit(0))
    ).alias("PRODUCT_VIEWS"),
    F.sum(
        F.when(F.col("EVENT_TYPE") == F.lit("cart_add"), F.lit(1))
        .otherwise(F.lit(0))
    ).alias("CART_ADDS")
])

engagement_df = engagement_df.with_column(
    "EMAIL_ENGAGEMENT_RATE",
    F.div0(F.col("EMAIL_CLICKS"), F.col("EMAIL_OPENS"))
)

print("Engagement feature DataFrame:")
engagement_df.show(5)

In [ ]:
engagement_fv = FeatureView(
    name="ENGAGEMENT_FEATURES",
    entities=[customer_entity],
    feature_df=engagement_df,
    refresh_freq="1 day",
    desc="Customer engagement and interaction features"
).attach_feature_desc({
    "TOTAL_INTERACTIONS": "Total count of all customer interactions",
    "WEBSITE_VISITS": "Number of website visits",
    "EMAIL_OPENS": "Number of emails opened",
    "EMAIL_CLICKS": "Number of email links clicked",
    "SUPPORT_TICKETS": "Number of support tickets created",
    "PRODUCT_VIEWS": "Number of product views",
    "CART_ADDS": "Number of items added to cart",
    "EMAIL_ENGAGEMENT_RATE": "Email click-through rate (clicks / opens)"
})

engagement_fv_registered = fs.register_feature_view(
    feature_view=engagement_fv,
    version="1.0",
    block=True
)

print("✓ Registered ENGAGEMENT_FEATURES feature view")

## Feature View 4: Derived Features

**Advanced derived features** computed from base features:
- **RFM Scores**: Quintile-based scoring for Recency, Frequency, Monetary
- **Lifecycle Stage**: Rule-based customer segmentation
- **Velocity Indicators**: Purchase and spending trends (30d vs 90d)
- **Engagement Ratios**: Engagement per purchase, support intensity
- **Cohort Comparisons**: Performance relative to tenure cohort

These features are computed in SQL/Snowpark for consistency between training and inference.

In [ ]:
# Join all base feature views to create derived features
# Use .fully_qualified_name() to get table names with version suffixes, then query them
rfm_table = rfm_fv_registered.fully_qualified_name()
purchase_table = purchase_patterns_fv_registered.fully_qualified_name()
engagement_table = engagement_fv_registered.fully_qualified_name()

rfm_base = session.table(rfm_table)
purchase_base = session.table(purchase_table)
engagement_base = session.table(engagement_table)

# Join all base features
base_features_df = rfm_base.join(
    purchase_base,
    on="CUSTOMER_ID",
    how="inner"
).join(
    engagement_base,
    on="CUSTOMER_ID",
    how="inner"
)

# 1. RFM Score (using NTILE for quintiles)
# NOTE: NTILE requires sorting within window, lower recency = better (reverse)
# Recency score: 5 (most recent) to 1 (least recent)
derived_df = base_features_df.with_column(
    "RECENCY_SCORE",
    (F.lit(6) - F.ntile(5).over(Window.order_by(F.col("RECENCY_DAYS"))))
)

# Frequency score: 1 (lowest) to 5 (highest)
derived_df = derived_df.with_column(
    "FREQUENCY_SCORE",
    F.ntile(5).over(Window.order_by(F.col("FREQUENCY")))
)

# Monetary score: 1 (lowest) to 5 (highest)
derived_df = derived_df.with_column(
    "MONETARY_SCORE",
    F.ntile(5).over(Window.order_by(F.col("MONETARY_TOTAL")))
)

# Composite RFM score (weighted average)
derived_df = derived_df.with_column(
    "RFM_SCORE",
    (F.col("RECENCY_SCORE") * F.lit(0.4) + 
     F.col("FREQUENCY_SCORE") * F.lit(0.3) + 
     F.col("MONETARY_SCORE") * F.lit(0.3))
)

# 2. Lifecycle Stage (rule-based segmentation)
# Use ntile(4) to calculate 75th percentile threshold for high_value customers
derived_df_with_monetary_quartile = derived_df.with_column(
    "MONETARY_QUARTILE",
    F.ntile(4).over(Window.order_by(F.col("MONETARY_TOTAL")))
)

derived_df = derived_df_with_monetary_quartile.with_column(
    "LIFECYCLE_STAGE",
    F.when(F.col("CUSTOMER_TENURE_DAYS") < F.lit(180), F.lit("new"))
    .when(F.col("RECENCY_DAYS") > F.lit(90), F.lit("at_risk"))
    .when(F.col("FREQUENCY") >= F.lit(20), F.lit("champion"))
    .when(F.col("MONETARY_QUARTILE") == F.lit(4), F.lit("high_value"))
    .otherwise(F.lit("regular"))
)

# Remove the helper column
derived_df = derived_df.drop("MONETARY_QUARTILE")

# 3. Purchase Consistency (placeholder - would need stddev of inter-purchase intervals)
derived_df = derived_df.with_column(
    "PURCHASE_CONSISTENCY",
    F.lit(1.0)  # Default to consistent (requires more complex calculation with transaction-level data)
)

# 4. Velocity Indicators
derived_df = derived_df.with_column(
    "PURCHASE_VELOCITY_30D",
    (F.col("RECENT_30D_COUNT") / F.lit(30))
)

derived_df = derived_df.with_column(
    "PURCHASE_VELOCITY_90D",
    (F.col("RECENT_90D_COUNT") / F.lit(90))
)

derived_df = derived_df.with_column(
    "SPENDING_VELOCITY_30D",
    (F.col("RECENT_30D_AMOUNT") / F.lit(30))
)

derived_df = derived_df.with_column(
    "SPENDING_VELOCITY_90D",
    (F.col("RECENT_90D_AMOUNT") / F.lit(90))
)

# Velocity acceleration
derived_df = derived_df.with_column(
    "VELOCITY_ACCELERATION",
    F.col("PURCHASE_VELOCITY_30D") - F.col("PURCHASE_VELOCITY_90D")
)

# 5. Engagement Ratios
derived_df = derived_df.with_column(
    "ENGAGEMENT_PER_PURCHASE",
    F.div0(F.col("TOTAL_INTERACTIONS"), F.col("FREQUENCY"))
)

derived_df = derived_df.with_column(
    "SUPPORT_INTENSITY",
    F.div0(F.col("SUPPORT_TICKETS"), F.col("FREQUENCY"))
)

# 6. Tenure Cohort (binning customer tenure)
derived_df = derived_df.with_column(
    "TENURE_COHORT",
    F.when(F.col("CUSTOMER_TENURE_DAYS") < F.lit(180), F.lit("0_6m"))
    .when(F.col("CUSTOMER_TENURE_DAYS") < F.lit(365), F.lit("6_12m"))
    .when(F.col("CUSTOMER_TENURE_DAYS") < F.lit(540), F.lit("12_18m"))
    .otherwise(F.lit("18m_plus"))
)

# 7. Cohort-based comparisons
# Calculate cohort averages
cohort_avg_monetary = F.avg("MONETARY_TOTAL").over(Window.partition_by("TENURE_COHORT"))
cohort_avg_frequency = F.avg("FREQUENCY").over(Window.partition_by("TENURE_COHORT"))

derived_df = derived_df.with_column(
    "MONETARY_VS_COHORT",
    F.div0(F.col("MONETARY_TOTAL"), cohort_avg_monetary)
)

derived_df = derived_df.with_column(
    "FREQUENCY_VS_COHORT",
    F.div0(F.col("FREQUENCY"), cohort_avg_frequency)
)

# Select only CUSTOMER_ID and derived features (drop base features to avoid duplication)
derived_features_only = derived_df.select([
    "CUSTOMER_ID",
    "RECENCY_SCORE",
    "FREQUENCY_SCORE",
    "MONETARY_SCORE",
    "RFM_SCORE",
    "LIFECYCLE_STAGE",
    "PURCHASE_CONSISTENCY",
    "PURCHASE_VELOCITY_30D",
    "PURCHASE_VELOCITY_90D",
    "SPENDING_VELOCITY_30D",
    "SPENDING_VELOCITY_90D",
    "VELOCITY_ACCELERATION",
    "ENGAGEMENT_PER_PURCHASE",
    "SUPPORT_INTENSITY",
    "TENURE_COHORT",
    "MONETARY_VS_COHORT",
    "FREQUENCY_VS_COHORT"
])

print("Derived features DataFrame:")
derived_features_only.show(5)

In [ ]:
derived_fv = FeatureView(
    name="DERIVED_FEATURES",
    entities=[customer_entity],
    feature_df=derived_features_only,
    refresh_freq="1 day",
    desc="Advanced derived features: RFM scores, lifecycle stages, velocity indicators, cohort comparisons"
).attach_feature_desc({
    "RECENCY_SCORE": "Recency quintile score (5=most recent, 1=least recent)",
    "FREQUENCY_SCORE": "Frequency quintile score (5=highest, 1=lowest)",
    "MONETARY_SCORE": "Monetary quintile score (5=highest, 1=lowest)",
    "RFM_SCORE": "Weighted composite RFM score (40% recency, 30% frequency, 30% monetary)",
    "LIFECYCLE_STAGE": "Customer lifecycle segmentation (new, at_risk, champion, high_value, regular)",
    "PURCHASE_CONSISTENCY": "Purchase timing consistency (higher = more predictable)",
    "PURCHASE_VELOCITY_30D": "Daily purchase rate over last 30 days",
    "PURCHASE_VELOCITY_90D": "Daily purchase rate over last 90 days",
    "SPENDING_VELOCITY_30D": "Daily spending rate over last 30 days",
    "SPENDING_VELOCITY_90D": "Daily spending rate over last 90 days",
    "VELOCITY_ACCELERATION": "Change in purchase velocity (30d vs 90d)",
    "ENGAGEMENT_PER_PURCHASE": "Average interactions per purchase",
    "SUPPORT_INTENSITY": "Average support tickets per purchase",
    "TENURE_COHORT": "Customer tenure bin (0_6m, 6_12m, 12_18m, 18m_plus)",
    "MONETARY_VS_COHORT": "Monetary total relative to tenure cohort average",
    "FREQUENCY_VS_COHORT": "Frequency relative to tenure cohort average"
})

derived_fv_registered = fs.register_feature_view(
    feature_view=derived_fv,
    version="1.0",
    block=True
)

print("✓ Registered DERIVED_FEATURES feature view")
print(f"  Features: {len(derived_features_only.columns) - 1}")  # -1 for CUSTOMER_ID

## Calculate Target Variable and Add to Customer Profile

**⚠️ IMPORTANT NOTE ON REAL-WORLD IMPLEMENTATION:**

In this demonstration, we calculate `FUTURE_12M_LTV` using a synthetic formula based on current features. This is necessary because we're working with simulated data.

**In a real-world production system, you would:**
1. **Use actual historical data**: Calculate the target by looking back at what customers *actually* spent in the 12 months following the observation date
2. **Example**: If observation_date = 2023-06-30, you'd sum all transactions from 2023-07-01 to 2024-06-30
3. **SQL approach**: `SELECT customer_id, SUM(amount) as future_12m_ltv FROM transactions WHERE transaction_date BETWEEN observation_date + 1 AND observation_date + 365 GROUP BY customer_id`
4. **This requires waiting**: You need 12 months of future data before you can train the model
5. **For inference**: You predict for the *current* period where you don't yet know the future value

**Key difference**: Real targets come from future observed behavior, not formulas. The formula here is only for demonstration purposes.

In [ ]:
# Calculate synthetic target variable
# (In real-world, you'd query historical future transactions directly)

transactions_df = session.table(f"{DATABASE}.{SCHEMA}.CONTINUOUS_TRANSACTIONS")
interactions_df = session.table(f"{DATABASE}.{SCHEMA}.CONTINUOUS_INTERACTIONS")

# Calculate basic RFM metrics needed for target calculation
rfm_for_target = transactions_df.group_by("CUSTOMER_ID").agg([
    F.datediff("day", F.max("TRANSACTION_DATE"), F.lit(OBSERVATION_DATE)).alias("RECENCY_DAYS"),
    F.count("TRANSACTION_ID").alias("FREQUENCY"),
    F.sum("AMOUNT").alias("MONETARY_TOTAL")
])

# Calculate total interactions
interactions_count = interactions_df.group_by("CUSTOMER_ID").agg(
    F.count("INTERACTION_ID").alias("TOTAL_INTERACTIONS")
)

# Join RFM and interactions with customer profile
customers_df = session.table(f"{DATABASE}.{SCHEMA}.CONTINUOUS_CUSTOMERS_PROFILE")

customers_with_metrics = customers_df.join(
    rfm_for_target, 
    on="CUSTOMER_ID", 
    how="left"
).join(
    interactions_count,
    on="CUSTOMER_ID",
    how="left"
)

# Calculate synthetic target variable (simulating what future 12 months would bring)
# In production: This would be SUM(actual_transactions) from next 12 months
customers_with_target = customers_with_metrics.with_column(
    "FUTURE_12M_LTV",
    (
        F.col("MONETARY_TOTAL") * F.lit(0.6) *
        F.greatest(F.lit(0.5), (F.lit(1.5) - F.col("RECENCY_DAYS") / F.lit(180))) *
        F.least(F.lit(2.0), (F.lit(1) + F.col("FREQUENCY") / F.lit(20))) *
        (F.lit(1) + F.col("TOTAL_INTERACTIONS") / F.lit(500)) *
        F.uniform(F.lit(0.7), F.lit(1.3), F.random())
    )
)

# Select just customer profile columns + target
customers_profile_with_target = customers_with_target.select([
    "CUSTOMER_ID",
    "SIGNUP_DATE", 
    "AGE_GROUP",
    "REGION",
    "SEGMENT",
    "HISTORY_DAYS",
    "FUTURE_12M_LTV"
])

# Save enhanced customer profile
customers_profile_with_target.write.mode("overwrite").save_as_table("CONTINUOUS_CUSTOMERS_PROFILE_WITH_TARGET")

print(f"✓ Saved customer profile with target: {customers_profile_with_target.count()} rows")

stats_df = session.table("CONTINUOUS_CUSTOMERS_PROFILE_WITH_TARGET").select(
    F.avg("FUTURE_12M_LTV").alias("AVG_LTV"),
    F.median("FUTURE_12M_LTV").alias("MEDIAN_LTV"),
    F.min("FUTURE_12M_LTV").alias("MIN_LTV"),
    F.max("FUTURE_12M_LTV").alias("MAX_LTV")
).collect()[0]

print(f"\nTarget variable statistics:")
print(f"  Average: ${stats_df['AVG_LTV']:.2f}")
print(f"  Median:  ${stats_df['MEDIAN_LTV']:.2f}")
print(f"  Min:     ${stats_df['MIN_LTV']:.2f}")
print(f"  Max:     ${stats_df['MAX_LTV']:.2f}")

## Generate Training Dataset from Feature Store

Now we use the enhanced customer profile (with target) as the spine and join **ALL** features from the Feature Store, including the new DERIVED_FEATURES.

In [ ]:
# Use customer profile with target as spine
spine_df = session.table(f"{DATABASE}.{SCHEMA}.CONTINUOUS_CUSTOMERS_PROFILE_WITH_TARGET")

print(f"Spine DataFrame: {spine_df.count()} customers with target variable")
spine_df.show(5)

In [ ]:
print("All registered feature views:")
fs.list_feature_views(entity_name="CUSTOMER").show()

## Summary

This notebook demonstrated:

1. **Feature Store Setup**: Created feature store and registered CUSTOMER entity
2. **Feature Views**: Created 4 managed feature views:
   - **RFM_FEATURES** (base metrics: recency, frequency, monetary, tenure)
   - **PURCHASE_PATTERNS** (behavioral patterns: categories, recent windows)
   - **ENGAGEMENT_FEATURES** (non-purchase interactions: website, email, support)
   - **DERIVED_FEATURES** ⭐ (computed features: RFM scores, lifecycle stages, velocities, cohort comparisons)
3. **Target Variable Creation**: Calculated FUTURE_12M_LTV and added to customer profile
   - ⚠️ In production: Would use actual historical future transactions, not formulas
4. **Training Dataset**: Generated training data combining customer profile + **ALL** features from **ALL** 4 feature views
5. **Benefits**:
   - ✓ **ALL feature logic centralized in Feature Store** - no computation in training code!
   - ✓ Automatic refresh (1 day schedule)
   - ✓ Reusable for training and inference
   - ✓ Feature lineage and discovery
   - ✓ **Zero training-serving skew** - same features everywhere

**Output Tables:**
- `CONTINUOUS_CUSTOMERS_PROFILE_WITH_TARGET`: Customer profiles with target variable
- `CONTINUOUS_TRAINING_DATA_WITH_TARGET`: Complete training dataset (profile + all 4 feature views + target)

**Key Improvement:**
All derived features (RFM scores, lifecycle stages, velocities, etc.) are now computed in the Feature Store, NOT in training code. This means:
- ✓ Training uses Feature Store features directly
- ✓ Inference uses the SAME Feature Store features
- ✓ Dynamic Tables can query Feature Views and call MODEL!PREDICT() with all features
- ✓ Guaranteed consistency - single source of truth!

**Next Steps**:
- Use `CONTINUOUS_TRAINING_DATA_WITH_TARGET` for model training (no feature computation needed!)
- Features automatically refresh daily from raw data
- Create Dynamic Table for automatic SQL-based inference